# 更快的索引

---

注：本课的理解需要其他知识作为补充，请查看文件：聚类算法、倒排表、数据查询.md; <br/>完整实现流程在文件：2. 更快的索引.py中

In [60]:
# 先测试一下使用暴力索引的方式，使用的时间
import numpy as np
import faiss
import time

In [61]:
# 创建原始数据
data = np.random.rand(100_000, 256)
dim = 256
index = faiss.IndexFlatL2(dim)
index.add(data)

In [62]:
# 创建查询并计算时间
query = np.random.rand(2, 256)

start_time = time.time()
D, I = index.search(query, k=2)
print(f"程序运行的时间是:{(time.time()-start_time):.4f}s")
print(D)
print(I)

程序运行的时间是:0.0062s
[[31.442684 32.060856]
 [30.78328  30.99403 ]]
[[18267 67958]
 [72973 40792]]


IndexIVFFlat函数所需参数（其中Flat指的是数据是使用未经压缩的形式存储的）：<br/>
1. 量化器(quantizer)：在聚类模型训练与数据查询时，选择怎样的相似度计算方式
2. 向量维度(d)：传入向量的维度
3. 聚类中心数量(nlist)：在聚类算法训练的时候需要找到几个聚类中心

---

除此之外，index还允许设置`index.nprobe=<数量>`来规定数据查询时，选择在几个最相似的簇中执行查询

In [63]:
# 使用IndexIDFFlat的方式创建索引并查询
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, 1000)
assert not index.is_trained  # 由于存在聚类模型，所以需要进行训练
index.train(data)  # 传入数据，找到规定数量的聚类中心
assert index.is_trained

In [64]:
# 创建新向量数据库
index.add(data)

In [65]:
# 执行查询，假设先在最近的一个簇中寻找数据，看看与暴力搜索的区别
index.nprobe = 1
start_time = time.time()
D, I = index.search(query, k=2)
print(f"程序运行的时间是:{(time.time()-start_time):.4f}s")
print(D)
print(I)

程序运行的时间是:0.0030s
[[31.442684 33.830658]
 [31.486443 31.658401]]
[[18267 73753]
 [87905  1463]]


我们不妨来对比一下，暴力搜索和IndexIVFFlat的结果差异:<br/>
**暴力搜索：** <br/>
程序运行的时间是:0.0062s <br/>
[31.442684 32.060856]<br/>
[30.78328  30.99403 ]<br/>
[18267 67958]<br/>
[72973 40792]<br/>

 **聚类倒排表搜索：**<br/>
程序运行的时间是:0.0030s<br/>
[31.442684 33.830658]<br/>
[31.486443 31.658401]<br/>
[18267 73753]<br/>
[87905  1463]<br/>

---

我们不难发现：效率确实提高了，但是精度降低了，暴力搜索得到的第二个最近距离为：32.060856小于新方法得到的：33.8360658<br/>
这就是因为"分簇搜索"，导致最优的结果可能被忽略了而没有找到，在实际运用中，需要通过交叉验证的方式平衡参数：nlist与nprobe，达到效率和精度的平衡
